In [116]:
import numpy as np
import keras.applications
from keras.applications.resnet50 import preprocess_input as p_i, decode_predictions, ResNet50
from keras.preprocessing import image
import h5py
from keras.activations import softmax, relu, sigmoid
#from keras.applications.vgg19 import VGG19
#from keras.applications.vgg19 import preprocess_input
import h5py
from PIL import Image, ImageOps
import gdal
import keras.backend as K
from keras.layers import Dense, Input, Conv2D, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers.core import Dropout, Activation
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.models import Model
import tensorflow as tf

import csv

In [117]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [118]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [119]:
def resize(path, size=224):
    ds = gdal.Open(path)
    im = np.nan_to_num(ds.ReadAsArray())
    im = (im * 255 / np.max(im)).astype('uint8')
    img = Image.fromarray(np.stack(im, axis=2))
    ratio = float(size/max(img.size))
    img = img.resize((int(ratio * img.size[0]), int(ratio * img.size[1])))
    return img

In [120]:
def resize_and_pad(path, size=224):
    img = resize(path, size)
    delta_w = size - img.size[0]
    delta_h = size - img.size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    img = ImageOps.expand(img, padding)
    return img

In [121]:
def create_model(input_shape, top='flatten'):
    if top not in ('flatten', 'avg', 'max'):
        raise ValueError('unexpected top layer type: %s' % top)

    # connects base model with new "head"
    BottleneckLayer = {
        'flatten': Flatten(),
        'avg': GlobalAveragePooling2D(),
        'max': GlobalMaxPooling2D()
    }[top]

    base = ResNet50(input_shape=input_shape,
                             include_top=False, 
                             weights='imagenet')
    print("here")
    #base.load_weights("imnet.hdf5")
    #print("weights done")
    x = BottleneckLayer(base.output)
    x = Dense(1, activation='linear')(x)
    model = Model(inputs=base.inputs, outputs=x)
    return model

In [100]:
#tempmodel = create_model((222, 222, 3))
model = create_model((224, 224, 3))

here


In [46]:
tempmodel.layers.pop(0)
inputs = Input(shape=(889, 889, 3)) #TODO: change to 899 later and fix data i'm so dumb
x = Conv2D(3, 5, strides=(4, 4))(inputs)
x = tempmodel(x)

In [34]:
newm = Model(inputs=inputs, outputs=x)
newm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 889, 889, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 3)       228       
_________________________________________________________________
model_3 (Model)              (None, 1)                 23688065  
Total params: 23,688,293
Trainable params: 23,635,173
Non-trainable params: 53,120
_________________________________________________________________


In [137]:
dummy = np.zeros((10, 889, 889, 3))

In [79]:
opt = SGD(lr=1e-5, decay=1e-6)
newm.compile(optimizer=opt, loss='mse')

In [140]:
newm.predict(dummy)

array([[3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09],
       [3.3863918e+09]], dtype=float32)

In [16]:
path = '/datasets/data/infra_ims'

In [15]:
import os

In [23]:
y_dict = {}
with open('data/nuts3density2014.csv', mode='r') as infile:
    reader = csv.reader(infile)
    y_dict = {rows[1]:rows[2] for rows in reader}

In [70]:
def im_to_arr(path):
    ds = gdal.Open(path)
    arr = np.nan_to_num(ds.ReadAsArray())
    return arr

In [17]:
paths = sorted(os.listdir(path))

In [18]:
for filename in paths:
    print(filename)
dal   

AT111_2014_infra.tif
AT112_2014_infra.tif
AT113_2014_infra.tif
AT121_2014_infra.tif
AT122_2014_infra.tif
AT123_2014_infra.tif
AT124_2014_infra.tif
AT125_2014_infra.tif
AT126_2014_infra.tif
AT127_2014_infra.tif
AT130_2014_infra.tif
AT211_2014_infra.tif
AT212_2014_infra.tif
AT213_2014_infra.tif
AT221_2014_infra.tif
AT222_2014_infra.tif
AT223_2014_infra.tif
AT224_2014_infra.tif
AT225_2014_infra.tif
AT226_2014_infra.tif
AT311_2014_infra.tif
AT312_2014_infra.tif
AT313_2014_infra.tif
AT314_2014_infra.tif
AT315_2014_infra.tif
AT321_2014_infra.tif
AT322_2014_infra.tif
AT323_2014_infra.tif
BG311_2014_infra.tif
BG344_2014_infra.tif
BG411_2014_infra.tif
BG412_2014_infra.tif
BG413_2014_infra.tif
BG414_2014_infra.tif
BG415_2014_infra.tif
BG421_2014_infra.tif
BG422_2014_infra.tif
BG423_2014_infra.tif
BG424_2014_infra.tif
BG425_2014_infra.tif
CZ010_2014_infra.tif
CZ020_2014_infra.tif
CZ031_2014_infra.tif
CZ032_2014_infra.tif
CZ041_2014_infra.tif
CZ042_2014_infra.tif
CZ051_2014_infra.tif
CZ052_2014_in

NameError: name 'dal' is not defined

In [38]:
from tqdm import tqdm
def generate_data(filepaths, path_name):
    X = []
    for filename in tqdm(filepaths):
        a = resize_and_pad(os.path.join(path_name, filename))
        X.append(np.array(a))
    return X

In [39]:
X = generate_data(paths, path)

100%|██████████| 393/393 [24:04<00:00,  3.68s/it]


In [58]:
X_224 = []
for x in X_three:
    img = Image.fromarray(x)
    ratio = float(224/max(img.size))
    img = img.resize((int(ratio * img.size[0]), int(ratio * img.size[1])))
    X_224.append(np.array(img))

In [62]:
np.save("X_224.npy", X_224)

In [64]:
X_three = [x[:,:,:3] for x in X_npy]
X_three = np.array(X_three)
X_three.shape

(393, 889, 889, 3)

In [28]:
X_three = np.load("data/X_three_889.npy")

In [5]:
y_npy = np.load("data/y_density.npy")

In [31]:
np.max(X_div[0])

0.9921568627450981

In [103]:

model.compile(optimizer=opt, loss='mse')

In [80]:
model.fit(X_224, ylog, epochs=50)

Epoch 1/50
393/393 [==============================] - 10s 25ms/step - loss: 3.8205
Epoch 2/50
393/393 [==============================] - 3s 7ms/step - loss: 3.6576
Epoch 3/50
393/393 [==============================] - 3s 7ms/step - loss: 3.5840
Epoch 4/50
393/393 [==============================] - 3s 7ms/step - loss: 3.8067
Epoch 5/50
393/393 [==============================] - 3s 7ms/step - loss: 3.5933
Epoch 6/50
393/393 [==============================] - 3s 7ms/step - loss: 3.8113
Epoch 7/50
393/393 [==============================] - 3s 7ms/step - loss: 3.7723
Epoch 8/50
393/393 [==============================] - 3s 7ms/step - loss: 4.0102
Epoch 9/50
393/393 [==============================] - 3s 7ms/step - loss: 3.9017
Epoch 10/50
393/393 [==============================] - 3s 7ms/step - loss: 3.8224
Epoch 11/50
393/393 [==============================] - 3s 7ms/step - loss: 3.9290
Epoch 12/50
393/393 [==============================] - 3s 7ms/step - loss: 3.8000
Epoch 13/50
393/393 [==

KeyboardInterrupt: 

In [123]:
model.predict(X_224[:5])

array([[-20.710573 ],
       [-19.22248  ],
       [ -4.143146 ],
       [  4.0577025],
       [ 16.204266 ]], dtype=float32)

In [124]:
ylog[:5]

array([3.99083419, 4.60816569, 4.20618404, 4.29456061, 4.33073334])

# Testing

In [146]:
model2 = create_model((224, 224, 3))

here


In [148]:
opt = SGD(lr=1e-7, decay=1e-6)
model2.compile(optimizer=opt, loss='mse')

In [130]:
X_small = X_224[:50]
y_small = ylog[:50]

In [170]:
model2.fit(X_small, y_small, epochs=50)

Epoch 1/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0052
Epoch 2/50
10/10 [==============================] - 0s 12ms/step - loss: 0.0051
Epoch 3/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0050
Epoch 4/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0050
Epoch 5/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0049
Epoch 6/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0048
Epoch 7/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0048
Epoch 8/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0047
Epoch 9/50
10/10 [==============================] - 0s 14ms/step - loss: 0.0047
Epoch 10/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0046
Epoch 11/50
10/10 [==============================] - 0s 12ms/step - loss: 0.0045
Epoch 12/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0045
Epoch 13/50
10/10 [==================

In [145]:
model.evaluate(X_224[10:], ylog[10:])

383/383 [==============================] - 8s 22ms/step


11.907031087900267

In [171]:
model2.predict(X_224[50:60])

array([[ 1.6926755 ],
       [ 0.02974572],
       [ 0.56700057],
       [-0.22696781],
       [ 0.76184356],
       [ 5.0818806 ],
       [ 0.42683157],
       [ 2.040916  ],
       [ 0.69119257],
       [ 0.23649588]], dtype=float32)

In [172]:
ylog[50:60]

array([5.11439453, 4.80484038, 5.00863329, 5.43633866, 6.6394836 ,
       4.52720864, 4.41763506, 6.40241459, 5.99545657, 4.29182837])

In [173]:
model2.predict(X_224[:10])

array([[4.0773425],
       [4.610155 ],
       [4.3328876],
       [4.1932945],
       [4.185093 ],
       [4.853313 ],
       [3.8174763],
       [3.9140508],
       [4.688249 ],
       [5.346055 ]], dtype=float32)

In [174]:
ylog[:10]

array([3.99083419, 4.60816569, 4.20618404, 4.29456061, 4.33073334,
       4.81543111, 3.8691155 , 3.94352167, 4.77322377, 5.42495002])

# Datagen stuff here

In [90]:
from keras.preprocessing.image import ImageDataGenerator

In [91]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [92]:
datagen.fit(X_224)


In [113]:
opt = SGD(lr=1e-9, decay=1e-6)

model.compile(optimizer=opt, loss='mse')

In [ ]:
epochs=20
model.fit_generator(datagen.flow(X_224, ylog, batch_size=32),
                    steps_per_epoch=len(X_224) / 32, epochs=epochs)

Epoch 1/20
13/12 [===============================] - 21s 2s/step - loss: 7.1333
Epoch 2/20
13/12 [===============================] - 5s 373ms/step - loss: 6.4525
Epoch 3/20
13/12 [===============================] - 5s 350ms/step - loss: 6.6590
Epoch 4/20
13/12 [===============================] - 4s 345ms/step - loss: 6.6352
Epoch 5/20
 8/12 [==================>...........] - ETA: 1s - loss: 6.6737

In [109]:
model.predict(X_224[10:15])

array([[  9.477249 ],
       [ 46.751263 ],
       [131.84785  ],
       [ -1.5433135],
       [ 27.43431  ]], dtype=float32)

In [110]:
ylog[10:15]

array([8.41438497, 4.96424225, 3.43075618, 3.81109709, 5.83947819])